## Initial code written for phase field simulation 
### Code borrowed from Mao et al. Soft Matter 2019
### Set to run conserved dynamics on a phase-field model coupled to a Flory-Huggins energy

In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#==============================================================================
# packages
#==============================================================================
# ohter standard libraries
import numpy as np
import numexpr as ne
import sys
import time 
sys.path.append('..')
from utils import convert_seconds_to_hms
import matplotlib.pyplot as plt

## FFT Solver object that does a semi-implicit-explicit calcuation for the derivative terms

In [29]:
# c = c0;
# cs = np.repeat(np.reshape(np.sum(c,axis=0)*-1+ 1,(1,64,64)) ,2,0);

# cHat = np.zeros_like(c0, dtype='complex')
# csHat = np.zeros_like(c0, dtype='complex')
# Solver1.fft(cs,csHat)
# Solver1.fft(c,cHat)
# print(csHat)

In [78]:
#==============================================================================
# solver object
#==============================================================================

class FFTSolver():
    
    def __init__(self, c_init, chiMat, lmbda, dt, T, N, start, root,kappa,kon,koff,chis,r):
        self.start = start
        self.root = root
        self.c0 = c_init.copy()
        self.NCom, _, _ = c_init.shape
        self.N = N
        self.gradMuX = c_init.copy()
        self.gradMuY = c_init.copy()
        self.gradcsX = c_init.copy()
        self.gradcsY = c_init.copy()

#         self.gradMuZ = c_init.copy()
        self.JXHat = np.zeros_like(c_init, dtype='complex')
        self.JYHat = np.zeros_like(c_init, dtype='complex')
        self.JsXHat = np.zeros_like(c_init, dtype='complex')
        self.JsYHat = np.zeros_like(c_init, dtype='complex')
        
#         self.JZHat = np.zeros_like(c_init, dtype='complex')
        self.chiMat = chiMat
        self.r = r
        self.chis = chis
        self.lmbda  = lmbda
        self.kappa = kappa
        self.dt     = dt
        self.kon    = kon
        self.koff   = koff
        self.T      = T
        self.x  = np.linspace(0, 1, N+1)[:-1]
        self.xx, self.yy = np.meshgrid(self.x, self.x)
        self.dx = 1.0/N
        self.k  = 2 * np.pi * np.fft.fftfreq(N, self.dx)
        self.kx = self.k.reshape(-1,1)
        self.ky = self.k.reshape(1,-1)
#         self.kz = self.k.reshape(1,1,-1)
        self.k2 = self.kx**2 + self.ky**2
        self.k4 = self.kx**4 + self.ky**4
        self.kxj = self.kx*1j
        self.kyj = self.ky*1j
#         self.kzj = self.kz*1j

    def fft(self, x, xHat):
        for i in range(self.NCom):
            xHat[i] = np.fft.fftn(x[i])


    def ifft(self, xHat, x):
        for i in range(self.NCom):
            x[i] = np.fft.ifftn(xHat[i]).real
            
        
    # calculate the chemical potential
    def cal_muHat(self, c, cHat):
        chiMat = self.chiMat
        k2 = self.k2
        lmbda = self.lmbda
        muHat = np.einsum('ij,jkl->ikl', chiMat, cHat*(1.0 - k2*lmbda))
        return muHat
    
    # calculate fluxes (multiplyied by Lij already)
    def cal_J(self, c, gradMu):
#         J0 = np.einsum('ijk,ijk->jk', c, gradMu)
        J  = ne.evaluate("c * (gradMu)")
        return J
    
    # calculate solvent fluxes (multiplyied by Lij already)
    def cal_sol_J(self, c, cs, gradMu):
        J  = ne.evaluate("(c * (gradMu))/cs")
        return J
    

    def cal_NHat(self, c, cHat, A):
        k2 = self.k2
        k4 = self.k4
        kxj = self.kxj
        kyj = self.kyj
#         kzj = self.kzj
        
        #solvent concentrations
        cs = np.repeat(np.reshape(np.sum(c,axis=0)*-1+ 1,(1,self.N,self.N)) ,self.NCom,0);
        csHat = np.zeros_like(c, dtype='complex')
        self.fft(cs,csHat)
        
        gradMuX = self.gradMuX
        gradMuY = self.gradMuY
        
        gradcsX = self.gradcsX
        gradcsY = self.gradcsY    
        
#         gradMuZ = self.gradMuZ
        JXHat = self.JXHat
        JYHat = self.JYHat
        JsXHat = self.JsXHat
        JsYHat = self.JsYHat

        #         JZHat = self.JZHat
        
        # diffsion part
        NDiff = - ne.evaluate("k2 * cHat")/self.r[:,None]
        
        # implicit part
        NImp  = ne.evaluate("A * k4 * cHat")
        
        # fluxes part
        lmbda = self.lmbda
        chiMat = self.chiMat
        chis = self.chis
#         cHatGrad = ne.evaluate("cHat*(1.0 - k2*lmbda)")
        cHatGrad = ne.evaluate("cHat*1.0")
        kappa_term = ne.evaluate("cHat*k2*lmbda")

        muHat = (chiMat.dot(cHatGrad.reshape(self.NCom, -1))).reshape(self.NCom,self.N, self.N) 
        muHat = muHat + (self.kappa.dot(kappa_term.reshape(self.NCom, -1))).reshape(self.NCom,self.N, self.N)
        muHat = muHat - (chis.dot(cHatGrad.reshape(self.NCom, -1))).reshape(self.NCom, self.N, self.N)
                
        chis_d = np.diag(chi_s[:,0])
        muHat  = muHat + (chis_d.dot(csHat.reshape(self.NCom, -1))).reshape(self.NCom, self.N, self.N)
        #muHat   = np.einsum('ij,jklm->iklm', chiMat, cHatGrad)
        
        # gradients -> depend on spatial dim
        muKxHat = ne.evaluate("kxj * muHat")
        self.ifft(muKxHat, gradMuX)
        self.ifft(ne.evaluate("kyj * muHat"), gradMuY)
#         self.ifft(ne.evaluate("kzj * muHat"), gradMuZ)

        # gradients --> from solvent flux terms
        csKxHat = ne.evaluate("kxj*csHat")
        
        self.ifft(csKxHat, gradcsX)
        self.ifft(ne.evaluate("kyj*csHat"), gradcsY)
        
        # calculate solvent fluxes
        JsX = self.cal_sol_J(c,cs, gradcsX)
        JsY = self.cal_sol_J(c,cs, gradcsY)
#         JZ = self.cal_J(c, gradMuZ)
        # fluxes contribution
        self.fft(JsX, JsXHat)
        self.fft(JsY, JsYHat)
#         self.fft(JZ, JZHat)
        NsFlux = ne.evaluate("kxj*JsXHat + kyj*JsYHat")

        # calculate fluxes:
        JX = self.cal_J(c, gradMuX)
        JY = self.cal_J(c, gradMuY)
#         JZ = self.cal_J(c, gradMuZ)
        # fluxes contribution
        self.fft(JX, JXHat)
        self.fft(JY, JYHat)
#         self.fft(JZ, JZHat)
        NFlux = ne.evaluate("kxj*JXHat + kyj*JYHat")
        return ne.evaluate("NFlux + NDiff + NImp - NsFlux")

    def save_images(self,c,step):
        colors = ["Greens","Oranges","Blues","Reds","Greys","Purples","PuRd","BuGn","Greens","Oranges","Blues","Reds","Greys","Purples","PuRd","BuGn"]

        levels = np.linspace(0.0, 1.0, 15)

        for i in np.arange(c.shape[0]):
            fig,ax = plt.subplots()
            cs = ax.contourf(c[i][:,:],cmap=plt.get_cmap(colors[i]),levels=levels)
            cbar = fig.colorbar(cs)
            fig.savefig(fname = self.root + 'c' + str(i) + '-' +str(self.start+step)+'.pdf',format='pdf',dpi=300)
            plt.close()
        
        print("Images saved at %d steps" % (step))
        
    def solve(self, outPrint, outSave):
        start = self.start
        root = self.root
        kon = self.kon
        koff = self.koff
        print(kon,koff)
    
        c = self.c0.copy()
        cHat = np.zeros_like(c, dtype='complex')
        self.fft(c, cHat)
        dt = self.dt
        delc_test = (kon- koff*c)*dt
        print(delc_test.max())
        T  = self.T
        tCur = 0.0
        A = 1.0 * self.chiMat.max() * self.lmbda
        #A = 0.0
        k4 = self.k4
        start_time = time.time()
        step = 0
        Ainv = 1.0/(1 + A * k4 * dt)
        print("--- Using %d of threads to calculate numpy fft---" % (ne.nthreads))
        while (tCur < T + dt/2.):
            tCur += dt
            step += 1
            cnHat = cHat
            cn = c
            ncHat = self.cal_NHat(cn, cnHat, A)
            cHat = ne.evaluate( "(cnHat + dt * ncHat) * Ainv" )
            self.ifft(cHat, c)
            c = c + (kon - koff*c)*dt/1.0;
            self.fft(c,cHat)

#             c = c + ne.evaluate("(kon - koff*c)*dt")
            if np.isnan(c.max()):
                print("Simulation ended because of NAN values")
                np.save(root + 'c-'+str(start + step)+'.npy', cn)
                break
            
            if (step%outPrint == 0):
                dtime = time.time() - start_time
                print("%d steps finished with max c %.4f, using %.4f seconds/step" % (step, c.max(), dtime/step))
                print(" Max change in concentration is %.4f, %4f" % (cn.max(),c.max()))

#                 if (start+step) < outSave:
#                     np.save(root + 'c-'+str(start+step)+'.npy', c)
#                     save_images(c,start,step,root)

            if (step%outSave == 0):
                print("----------------------step %d saved---------- ------------------" % (step+start))
                c_all_temp = np.concatenate((c,np.reshape(np.sum(c,axis=0)*-1+ 1,(1,N,N))),axis=0)
                self.save_images(c_all_temp,step)
                np.save(root + 'c-'+str(start+step)+'.npy', c_all_temp)
                with open(root+ "/stats.txt", 'a') as stats:
                    output_vars = [step,tCur,dt];
                    for i in range(c_all_temp.shape[0]):
                        output_vars.append(c_all_temp[i].max())
                        output_vars.append(c_all_temp[i].min())
                    output_vars.append(time.time() - start_time)
                    
                    stats.write("\t".join([str(it) for it in output_vars]) + "\n")

        total_time = time.time() - start_time
        hrs, mins, secs = convert_seconds_to_hms(total_time)
        print("--- total time = %d hrs %d mins %.4f secs ---" % (hrs, mins, secs))

    



# Runtime code

### Dependencies for runtime code

In [60]:
#==============================================================================
# packages
#==============================================================================

# ohter standard libraries
import numpy as np
import os
import sys
import time
import datetime



In [61]:
def gen_c0(c_init, M, N,noise_strength=0.01):
    c0_init = np.array(c_init)
    cmin = c0_init.min()
    noise_initial = noise_strength * cmin * np.random.uniform(-1,1,(M, N, N))
    noise_initial -= noise_initial.mean(0)
    c0 = c0_init[:, np.newaxis, np.newaxis] + noise_initial
    return c0


### Main runtime input

In [93]:
# if __name__ == '__main__':

# sys.path.append('..')
# from utils import equal_chiMat, non_uniform_mixutre, gen_c0

###################################################
#parameters go here
###################################################

# system argument (number of components other than solvent)
NCom  = 5
dim   = 2
# N     = int(sys.argv[1])
# start = int(sys.argv[2])
# end   = int(sys.argv[3])
# outfolder = str(sys.argv[4])
random_seed = np.random.randint(1e6)
random_seed = 245;
np.random.seed(seed=random_seed)

N     = 64
start = 0
end   = 2e4
outfolder = 'Output/test/'

steps = end - start

# physical parameters
lmbda = 5.0e-5
dt  = 1.0e-6
T   = steps * dt
beta =(NCom)/(NCom+1);

# chi (i,j) interaction matrix
chi_mean = 0;
chi_std =4.0
chi = chi_std*np.random.randn(NCom,NCom) + chi_mean;
np.fill_diagonal(chi, 0);
# print(chi[-1,0:-1])
# # chi[-1,:] = 0.0;
# # print(chi[-1,0:-1])
# chi[2,1] = -4.0

# chi(i,s) is vector of component solvent interactions
chi_s = np.zeros((1,NCom))
# chi_s += chi_mean;
# chi_s[0,-1] = 4.5;
chi_s = np.repeat(chi_s,NCom,0)
print(chi_s)

# polymer lengths is vector of polymerizations
r_mu = 1.0;
r_sigma = 1.0;
#     r = r_sigma*np.random.randn(N,1) +r_mu ;
#     r = r_sigma*np.random.lognormal(r_mu,r_sigma,size=(N,1)) ;

r = r_mu*np.random.geometric(p=r_sigma, size=(NCom,1))
#     print(np.mean(r))
r[r<1] = 1;
print(r)


g = np.zeros((NCom,NCom))

# Fill in chi and mobility matrices
np.fill_diagonal(g, beta/NCom)
for i in np.arange(len(chi)-1):
    for j in np.arange(i+1,len(chi)):
        chi[i,j] = chi[j,i]
#         g[i,j] = -(1/NCom**2)
#         g[j,i] = g[i,j]

print(chi)
J = chi+np.identity(NCom)*NCom/(beta*r) + 1/(1-beta)  - chi_s - chi_s.T
print(J)
wJ, vJ = np.linalg.eig(J)
print(min(wJ))
Jeff = np.matmul(g,J);
wJeff,vJeff = np.linalg.eig(Jeff)
print(min(wJeff))

kon = 0.0;
koff = kon*NCom/beta;
print(kon,koff)

chiMat = chi;
kappa = np.identity(NCom)*10.0
# chi01 = 3.00
# chi02 = 7.00
# chi12 = 4.00
# chi03 = 6.00



# output control
outPrint = 1e3
outSave  = 1e3


c_init = beta*np.ones((NCom))/NCom
c0 = gen_c0(c_init, NCom, N,noise_strength=0.5*beta/NCom)

print(c_init)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[ 0.         -3.32062674  5.72420445  4.13618781 -3.44423872]
 [-3.32062674  0.         -8.57141541 -4.62084436 -3.12795106]
 [ 5.72420445 -8.57141541  0.         -3.32775029 -2.95696971]
 [ 4.13618781 -4.62084436 -3.32775029  0.         -2.37398377]
 [-3.44423872 -3.12795106 -2.95696971 -2.37398377  0.        ]]
[[12.          2.67937326 11.72420445 10.13618781  2.55576128]
 [ 2.67937326 12.         -2.57141541  1.37915564  2.87204894]
 [11.72420445 -2.57141541 12.          2.67224971  3.04303029]
 [10.13618781  1.37915564  2.67224971 12.          3.62601623]
 [ 2.55576128  2.87204894  3.04303029  3.62601623 12.        ]]
-2.984051769946493
-0.49734196165774863
0.0 0.0
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


# initialize

In [94]:
end = 1e3
root =  outfolder + '/' + str(datetime.date.today()).replace('-','') + '/' + str(N) + '_' + str(start) + '_' + str(end) + '_' + str(NCom) + '_' + str(kon) + '/' + str(random_seed) + '/'
os.makedirs(root,exist_ok=True)
print(root)
np.save(root+'c-'+str(start)+'.npy', c0)
with open(root+ "/stats.txt", 'w+') as stats:
    str_header = ["step","t","dt"]
    for i in range(NCom+1):
        str_header.append("c"+str(i)+"_max")
        str_header.append("c"+str(i)+"_min")

    stats.write("\t".join(str_header) + "\n")
###################################################
#solving
###################################################

# from FFT_nV_3D import FFTSolver

Solver1 = FFTSolver(c_init=c0,
                    chiMat=chiMat, lmbda=lmbda,
                    dt=dt, T=T, N=N,
                    start=start, root=root,kappa=kappa,kon=kon,koff=koff,chis=chi_s,r=r)

Solver1.save_images(c0,start)
cFFT3 = Solver1.solve(outPrint=outPrint, outSave=outSave)
#np.save(root+'c-'+str(end + 1)+'.npy', cFFT3)

###################################################
#end
###################################################


Output/test//20210222/64_0_1000.0_5_0.0/245/
Images saved at 0 steps
0.0 0.0
0.0
--- Using 8 of threads to calculate numpy fft---
1000 steps finished with max c 0.1802, using 0.0073 seconds/step
 Max change in concentration is 0.1802, 0.180238
----------------------step 1000 saved---------- ------------------
Images saved at 1000 steps
2000 steps finished with max c 0.1899, using 0.0076 seconds/step
 Max change in concentration is 0.1899, 0.189882
----------------------step 2000 saved---------- ------------------
Images saved at 2000 steps
3000 steps finished with max c 0.2095, using 0.0077 seconds/step
 Max change in concentration is 0.2095, 0.209509
----------------------step 3000 saved---------- ------------------
Images saved at 3000 steps
4000 steps finished with max c 0.2479, using 0.0077 seconds/step
 Max change in concentration is 0.2479, 0.247870
----------------------step 4000 saved---------- ------------------
Images saved at 4000 steps
5000 steps finished with max c 0.3132,

In [13]:
import moviepy as mp

In [ ]:
def key_func(x):
    """
    Input   =   List of PNG image file paths (x)
    Output  =   List of filenames without extensions (to sort)
    """
    return int(x.split('_')[-1].rstrip('.png'))

def save_movie(dir_name,NCom,movie_name ='movie',duration=0.1):
    """
    Function generates 2D movies (gif & MP4) from output images
    **Input parameters**
        -   dir_name    =   path to directory with image files
        -   movie_name  =   prefix for output movie file (default = movie)
        -   duration    =   time between 2 frames (default = 0.1)
    """
    all_files = os.listdir(dir_name);
    for i in range(NCom+1):
        relevant_files = [f for f in all_files if (f.find('c'+str(i))>-1 and f.endswith('.png'))]
        relevant_files = [dir_name + f for f in sorted(relevant_files,key=key_func)]    
        clip = mp.ImageSequenceClip(relevant_files,fps=1/duration);
        clip.write_gif(dir_name + 'c' + str(i) +'-' +  movie_name + '.gif');

## Solver1.gradcsX

In [14]:
sim_output_files = os.listdir(root)
output_data = []
for f in sim_output_files:
    if f.find('.npy') > -1:
        output_data.append(np.load(root+f))
print(len(output_data))

51


In [16]:
c_final = output_data[-1]
c_final.shape

(4, 64, 64)

In [28]:
kon = 1.0;
koff= 4.0;
flux = ne.evaluate("(kon - koff*c_final)*dt")
dflux = ne.evaluate("")

In [31]:
c_final[0][1,1]- 0.25*(c_final[0][0,1] + c_final[0][1,0] + c_final[0][2,1] + c_final[0][1,2] )

-0.0003668471778335569

random_seed = np.random.randint(1e6)


###################################################
#initialize
###################################################

if start < 1:
#     c_init = np.ones((NCom))/NCom
    #c0 = non_uniform_mixutre(c_init, N=N, dim=dim)
    c0 = gen_c0(c_init, NCom, N)
else:
    c0 = np.load(root+'c-'+str(start)+'.npy')

In [ ]:
# od = output_data[-1].reshape((6,64*64))
# od.shape

In [ ]:
# c_all_temp = np.stack(output_data[-1],axis=-1)
c_all_temp = output_data[-1]

print(c_all_temp.shape)
c_all = np.reshape(c_all_temp,(5,4096*1))
c_all.shape

In [ ]:
covariance_matrix = np.cov(c_all)/c_all.shape[0]
v, w = np.linalg.eig(covariance_matrix)
idx = v.argsort()[::-1] # Sort descending and get sorted indices
v = v[idx] # Use indices on eigv vector
w = w[:,idx] # 
print("Eigenvalues: \n", v, "\n")

print("Eigenvector: \n",w,"\n")
variance_explained = []
for i in v:
     variance_explained.append((i/sum(v))*100)
        
print(variance_explained)

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd

In [ ]:
X_embedded = TSNE(n_components=2).fit_transform(c_all.T)
X_embedded.shape

In [ ]:
fig,axes = plt.subplots(1,1)
axes.scatter(X_embedded[:,0],X_embedded[:,1])
plt.show()

In [ ]:
sns.pairplot(pd.DataFrame(c_all.T))

In [ ]:
pd.DataFrame(c_all.T)

In [ ]:
g = sns.clustermap(c_all.T)

In [ ]:
fig,axes = plt.subplots(1,1)
axes.scatter(c_all.T.dot(w[:,0]),c_all.T.dot(w[:,2]))
plt.show()

## import matplotlib.pyplot as plt
# for i in np.arange(len(output_data)):
#     fig,ax = plt.subplots()
#     cs = ax.contourf(output_data[i][0][:,:,10],cmap=plt.get_cmap("Greens"),vmin = 0,vmax =1.0)
#     cbar = fig.colorbar(cs)
# plt.show()

J = chi + np.identity(NCom)*NCom
print(J)
print(np.linalg.eigvals(J))
Jeff = np.matmul(g,J)
print(np.linalg.eigvals(Jeff))
Jeff = np.matmul(J,g)
print(np.linalg.eigvals(Jeff))
Jeff = np.matmul(g,chi+np.identity(NCom)*NCom);
wJeff,vJeff = np.linalg.eig(Jeff)
print(wJeff)